In [1]:
from mindnlp.utils import http_get

url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
path = http_get(url, './')

/home/daiyuxin/anaconda3/envs/mindspore/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from mindspore.dataset import TextFileDataset

dataset = TextFileDataset(str(path), shuffle=False)
dataset.get_dataset_size()

50000

In [3]:
train_dataset, test_dataset = dataset.split([0.9, 0.1], randomize=False)

In [4]:
# article: [CLS] xxxxx [SEP]
# summary: [CLS] xxxxx [SEP]

In [5]:
import json
import numpy as np

def process_dataset(dataset, tokenizer, batch_size=6, max_seq_len=1024, shuffle=False):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def merge_and_pad(article, summary):
        tokenized = tokenizer(text=article, text_pair=summary,
                              padding='max_length', truncation='only_first', max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['input_ids']
    
    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(merge_and_pad, ['article', 'summary'], ['input_ids', 'labels'])

    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(batch_size)

    return dataset

In [6]:
from mindnlp.transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

/home/daiyuxin/anaconda3/envs/mindspore/lib/python3.9/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'modelscope.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/daiyuxin/anaconda3/envs/mindspore/lib/python3.9/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'modelscope.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/daiyuxin/anaconda3/envs/mindspore/lib/python3.9/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'modelscope.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warn

In [7]:
train_dataset = process_dataset(train_dataset, tokenizer, batch_size=4)

In [8]:
next(train_dataset.create_tuple_iterator())

[Tensor(shape=[4, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0],
  [ 101,  704, 3173 ...    0,    0,    0],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]]),
 Tensor(shape=[4, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0],
  [ 101,  704, 3173 ...    0,    0,    0],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]])]

In [9]:
len(tokenizer)

21128

In [10]:
from mindspore import ops
from mindnlp.transformers import GPT2LMHeadModel

class GPT2ForSummarization(GPT2LMHeadModel):
    def construct(
        self,
        input_ids = None,
        attention_mask = None,
        labels = None,
    ):
        outputs = super().construct(input_ids=input_ids, attention_mask=attention_mask)
        shift_logits = outputs.logits[..., :-1, :]
        shift_labels = labels[..., 1:]
        # Flatten the tokens
        loss = ops.cross_entropy(shift_logits.view(-1, shift_logits.shape[-1]), shift_labels.view(-1), ignore_index=tokenizer.pad_token_id)
        return loss

In [11]:
from mindspore import ops
from mindspore.nn.learning_rate_schedule import LearningRateSchedule

class LinearWithWarmUp(LearningRateSchedule):
    """
    Warmup-decay learning rate.
    """
    def __init__(self, learning_rate, num_warmup_steps, num_training_steps):
        super().__init__()
        self.learning_rate = learning_rate
        self.num_warmup_steps = num_warmup_steps
        self.num_training_steps = num_training_steps

    def construct(self, global_step):
        if global_step < self.num_warmup_steps:
            return global_step / float(max(1, self.num_warmup_steps)) * self.learning_rate
        return ops.maximum(
            0.0, (self.num_training_steps - global_step) / (max(1, self.num_training_steps - self.num_warmup_steps))
        ) * self.learning_rate

In [12]:
num_epochs = 5
warmup_steps = 2000
learning_rate = 1.5e-4

num_training_steps = num_epochs * train_dataset.get_dataset_size()

In [13]:
from mindspore import nn
from mindnlp.transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(vocab_size=len(tokenizer))
model = GPT2ForSummarization(config)

lr_scheduler = LinearWithWarmUp(learning_rate=learning_rate, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)
optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=lr_scheduler)

In [14]:
# 记录模型参数数量
print('number of model parameters: {}'.format(model.num_parameters()))

number of model parameters: 102068736


In [15]:
from mindnlp.engine import Trainer
from mindnlp.engine.callbacks import CheckpointCallback

ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt2_summarization',
                                epochs=1, keep_checkpoint_max=2)

trainer = Trainer(network=model, train_dataset=train_dataset,
                  epochs=5, optimizer=optimizer, callbacks=ckpoint_cb)
trainer.set_amp(level='O1')

Trainer will use 'StaticLossScaler' with `scale_value=2 ** 10` when `loss_scaler` is None.


In [16]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0:   0%|                                                                                                         | 0/11250 [00:00<?, ?it/s][ERROR] CORE(1809837,7f95ef9896c0,python):2023-11-26-01:46:08.184.627 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1809837/729418715.py]
[ERROR] CORE(1809837,7f95ef9896c0,python):2023-11-26-01:46:08.184.657 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1809837/729418715.py]
[ERROR] CORE(1809837,7f95ef9896c0,python):2023-11-26-01:46:08.184.892 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1809837/729418715.py]
[ERROR] CORE(1809837,7f95ef9896c0,python):2023-11-26-01:46:08.184.904 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1809837/729418715.py]
Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████| 11250/11250 [1:

Checkpoint: 'gpt2_summarization_epoch_0.ckpt' has been saved in epoch: 0.


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████| 11250/11250 [1:23:40<00:00,  2.24it/s, loss=4.7891397]


Checkpoint: 'gpt2_summarization_epoch_1.ckpt' has been saved in epoch: 1.


...

The maximum number of stored checkpoints has been reached.
Checkpoint: 'gpt2_summarization_epoch_2.ckpt' has been saved in epoch: 2.


Epoch 3: 100%|████████████████████████████████████████████████████████████████████████████| 11250/11250 [1:23:36<00:00,  2.24it/s, loss=4.789144]


The maximum number of stored checkpoints has been reached.
Checkpoint: 'gpt2_summarization_epoch_3.ckpt' has been saved in epoch: 3.


Epoch 4: 100%|███████████████████████████████████████████████████████████████████████████| 11250/11250 [1:23:36<00:00,  2.24it/s, loss=4.7891893]


The maximum number of stored checkpoints has been reached.
Checkpoint: 'gpt2_summarization_epoch_4.ckpt' has been saved in epoch: 4.


In [17]:
def process_test_dataset(dataset, tokenizer, batch_size=1, max_seq_len=1024, max_summary_len=100):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def pad(article):
        tokenized = tokenizer(text=article, truncation=True, max_length=max_seq_len-max_summary_len)
        return tokenized['input_ids']

    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(pad, 'article', ['input_ids'])
    
    dataset = dataset.batch(batch_size)

    return dataset

In [18]:
batched_test_dataset = process_test_dataset(test_dataset, tokenizer, batch_size=1)

In [19]:
print(next(batched_test_dataset.create_tuple_iterator(output_numpy=True)))

[array([[ 101, 3173, 1290, 5381, 6205, 2128,  128, 3299,  129, 3189, 4510,
        8020, 6381, 5442, 6948, 1132,  840, 8021, 6381, 5442,  129, 3189,
         678, 1286,  794, 7362, 6205, 4209,  689, 1265, 2339, 7415, 1730,
         749, 6237, 1168, 8024,  754,  128, 3299,  127, 3189, 7506, 1814,
        3433, 3409, 1790,  759,  678,  752, 3125,  704, 6158, 1737, 4638,
         125, 1399, 4771, 2339, 2347, 4802, 6371, 1059, 6956, 3647,  767,
         511,  128, 3299,  127, 3189, 1119, 3247,  125, 3198, 8123, 1146,
        8024, 7362, 6205, 4689, 7506, 1814, 3433, 3409, 1790, 4209, 4771,
        2963, 6822,  676, 7339, 1762, 4209, 4771,  122, 1384,  759, 1298,
        7023, 1277,  671, 2339,  868, 7481, 6822, 6121,  868,  689, 3198,
        1355, 4495,  671, 6629, 4209,  680, 4482, 3172, 4960, 1139,  752,
        3125, 8024,  125, 1399, 4771, 2339, 6158, 1737,  511, 3131, 3001,
         782, 1447,  754,  127, 3189,  678, 1286, 1355, 4385,  671, 1399,
        6158, 1737, 4771, 2339, 8024,

In [22]:
model = GPT2LMHeadModel.from_pretrained('./checkpoint/gpt2_summarization_epoch_4.ckpt', config=config)

/home/daiyuxin/anaconda3/envs/mindspore/lib/python3.9/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'modelscope.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Generation config file not found, using a generation config created from the model config.


In [ ]:
model.set_train(False)
model.config.eos_token_id = model.config.sep_token_id
i = 0
for (input_ids, raw_summary) in batched_test_dataset.create_tuple_iterator():
    output_ids = model.generate(input_ids, max_new_tokens=50, num_beams=5, no_repeat_ngram_size=2)
    output_text = tokenizer.decode(output_ids[0].tolist())
    print(output_text)
    i += 1
    if i == 10:
        break